In [1]:
import os
import sys
import math
import numpy as np
import netCDF4 as nc
from matplotlib import pyplot as plt
from math import exp, sqrt, log
import time
import geopandas as gpd
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
if '..' not in sys.path:
    sys.path.append('..')
from importlib import reload
#reload(util_tools)
import util_tools
from util_tools.data_loader import data_processer

C:\Users\96349\anaconda3\envs\Downscale_env\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\96349\anaconda3\envs\Downscale_env\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\96349\anaconda3\envs\Downscale_env\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\96349\anaconda3\env

# 1. Data Processing

In [2]:
# define necessary parameters
target_var = 'DUEXTTAU'
file_path_g_06 = r'C:\Users\96349\Documents\Downscale_data\MERRA2\G5NR_aerosol_variables_over_MiddleEast_daily_20060516-20070515.nc'
file_path_g_05 = r'C:\Users\96349\Documents\Downscale_data\MERRA2\G5NR_aerosol_variables_over_MiddleEast_daily_20050516-20060515.nc'
file_path_m = r'C:\Users\96349\Documents\Downscale_data\MERRA2\MERRA2_aerosol_variables_over_MiddleEast_daily_20000516-20180515.nc'
file_path_ele = r'C:\Users\96349\Documents\Downscale_data\elevation\elevation_data.npy'
file_path_country = [r'C:\Users\96349\Documents\Downscale_data\Country_shape\AFG_adm/AFG_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\ARE_adm/ARE_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\IRQ_adm/IRQ_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\KWT_adm/KWT_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\QAT_adm/QAT_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\SAU_adm/SAU_adm0.shp']
n_lag = 20
n_pred = 1
task_dim = [5, 5]

In [7]:
def plot_image(single_image_data, title, lats, lons, save_plt=False):
    '''
    if is_merra:
        lons = MERRA_data.variables['lon'][:]
        lats = MERRA_data.variables['lat'][:]
    else:
        lons = G5NR_data.variables['lon'][:]
        lats = G5NR_data.variables['lat'][:]
    '''
    m = Basemap(projection='merc',llcrnrlon=25.,llcrnrlat=10.,urcrnrlon=75.,urcrnrlat=45. , resolution='h', epsg = 4326)
    m.drawcoastlines()
    m.drawstates()
    m.drawcountries()
    parallels = np.arange(10,43,5.) # make latitude lines ever 5 degrees from 30N-50N
    meridians = np.arange(25,75,5.) # make longitude lines every 5 degrees from 95W to 70W
    m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
    m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
    #m.arcgisimage(service='ESRI_Imagery_World_2D', xpixels = 1500, verbose= True)
    m.shadedrelief(scale=0.5)
    m.pcolormesh(lons, lats, single_image_data, latlon=True)
    plt.clim(0, single_image_data.max())
    m.drawcoastlines(color='lightgray')
    plt.title(title)
    plt.colorbar()
    if save_plt:
        plt.savefig(title+'.jpg')

In [3]:
#reload(util_tools.data_loader)
import util_tools.data_loader as data_loader

In [4]:
data_processor = data_loader.data_processer()
g_data, m_data, [G_lats, G_lons, M_lats, M_lons], ele_data = data_processor.load_data(target_var, file_path_g_05, file_path_g_06, file_path_m, file_path_ele, file_path_country)

In [5]:
m_data = data_processor.unify_m_data(g_data[:10], m_data, G_lats, G_lons, M_lats, M_lons)

In [6]:
match_m_data = m_data[range(1826, 1826+730), :, :]

In [7]:
g_data.shape

(730, 360, 647)

In [8]:
X_high, X_low, X_ele, X_other, Y = data_processor.flatten(g_data[:50, :30, :30], match_m_data[:50, :30, :30], ele_data[:30, :30],
                                                          [G_lats[:30], G_lons[:30]], list(range(50)), n_lag=n_lag, n_pred=n_pred, task_dim=task_dim, is_perm=True, 
                                                          return_Y=True)

# 2. GAN model test

In [9]:
def nnelu(input):
    return tf.add(tf.constant(1, dtype=tf.float32), tf.nn.elu(input))

def mapping_to_target_range( x, target_min=0, target_max=1 ) :
    x02 = tf.keras.backend.tanh(x) + 1 # x in range(0,2)
    scale = ( target_max-target_min )/2.
    return  x02 * scale + target_min


high_input = tf.keras.Input(shape=(n_lag, task_dim[0], task_dim[1], 1))
x1 = tf.keras.layers.ConvLSTM2D(16, kernel_size=(3,3), return_sequences=True, activation=tf.keras.layers.LeakyReLU())(high_input)
#x1 = tf.keras.layers.ConvLSTM2D(16, kernel_size=(3,3), activation=tf.keras.layers.LeakyReLU())(x1)
x1 = tf.keras.layers.Flatten()(x1)

low_input = tf.keras.Input(shape=(n_lag, task_dim[0], task_dim[1], 1))
x2 = tf.keras.layers.ConvLSTM2D(16, kernel_size=(3,3), return_sequences=True, activation=tf.keras.layers.LeakyReLU())(low_input)
x2 = tf.keras.layers.Flatten()(x2)

ele_input = tf.keras.Input(shape=(task_dim[0], task_dim[1], 1))
x3 = tf.keras.layers.Conv2D(16, kernel_size=(3,3), activation=tf.keras.layers.LeakyReLU())(ele_input)
x3 = tf.keras.layers.Flatten()(x3)

other_input =  tf.keras.Input(shape=(3))
x4 = tf.keras.layers.Dense(8, activation=tf.keras.layers.LeakyReLU())(other_input)

x = tf.keras.layers.Concatenate(axis=1)([x1, x2, x3, x4])
#x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(n_pred*np.prod(task_dim), activation=nnelu)(x)
x = tf.keras.layers.Reshape([n_pred, task_dim[0], task_dim[1]])(x)
generator = tf.keras.Model([high_input, low_input, ele_input, other_input], x)
opt = tf.keras.optimizers.Adam(learning_rate=0.005)
generator.compile(optimizer=opt, loss='mean_absolute_error')

In [10]:
# define callbacks
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=1, factor=0.1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
best_save = tf.keras.callbacks.ModelCheckpoint('s2s_model', save_best_only=True, monitor='val_loss', mode='min')
callbacks = [lr_scheduler, early_stopping, best_save]

history = generator.fit([X_high[:-3], X_low[:-3], X_ele[:-3], X_other[:-3]], Y[:-3], epochs=1, callbacks=callbacks, validation_split=0.25)

475/476 [============================>.] - ETA: 0s - loss: 0.0074

INFO:tensorflow:Assets written to: s2s_model\assets


INFO:tensorflow:Assets written to: s2s_model\assets


476/476 [==============================] - 61s 110ms/step - loss: 0.0074 - val_loss: 0.0029 - lr: 0.0050


In [13]:
np.round(history.history['val_loss'][0], 5)

0.00288

In [19]:
generator.predict([X_high[-2:], X_low[-2:], X_ele[-2:], X_other[-2:]])

array([[[[0.        , 0.96585596, 0.96535116, 0.        , 0.9653614 ],
         [0.965236  , 0.96597326, 0.9651004 , 0.        , 0.96587306],
         [0.96600735, 0.9657643 , 0.9659777 , 0.9651386 , 0.9654685 ],
         [0.966095  , 0.96596116, 0.        , 0.96616995, 0.9652539 ],
         [0.9658393 , 0.9666162 , 0.        , 0.96587276, 0.9654851 ]]],


       [[[0.        , 0.9661002 , 0.96582234, 0.        , 0.9656026 ],
         [0.9651752 , 0.966246  , 0.9657142 , 0.        , 0.9659857 ],
         [0.965397  , 0.9660406 , 0.9659572 , 0.9658715 , 0.96645546],
         [0.9660278 , 0.9664916 , 0.        , 0.96617365, 0.96510905],
         [0.96557724, 0.9660495 , 0.        , 0.9666382 , 0.96577233]]]],
      dtype=float32)

In [16]:
Y[-2:]

array([[[[0.96209797, 0.9621877 , 0.96193371, 0.96189912, 0.96229232],
         [0.96203399, 0.96214366, 0.96209948, 0.9618981 , 0.96197329],
         [0.96180667, 0.96200721, 0.96211405, 0.96204762, 0.96190014],
         [0.96150141, 0.96180359, 0.96189352, 0.96187211, 0.9616579 ],
         [0.96111839, 0.96135704, 0.96149043, 0.96148572, 0.96143228]]],


       [[[0.96630322, 0.96626718, 0.96622566, 0.96619059, 0.96614191],
         [0.96634224, 0.96628981, 0.96623644, 0.96618365, 0.96611053],
         [0.96633115, 0.96626986, 0.96620216, 0.96613494, 0.96606159],
         [0.96629403, 0.96621372, 0.96615042, 0.96611402, 0.96609423],
         [0.96622451, 0.96617284, 0.96615003, 0.96614539, 0.96615042]]]])

In [20]:
g2_model = tf.keras.models.load_model('s2s_model')

In [27]:
pd.DataFrame(history.history).to_csv('history.csv')

In [26]:
pred_Y = g2_model.predict([X_high[-1:], X_low[-1:], X_ele[-1:], X_other[-1:]])

In [28]:
pred_Y.mean()

0.9427106

In [112]:
np.sqrt(np.mean(np.square(pred_Y[-1] - Y[-1])))

0.010288388300887755

In [100]:
from util_tools.data_processing import rsquared
rsquared(pred_Y[0, 0].reshape((25)), Y[0,0].reshape((25)))

(0.008515162367835552, 0.6608753429840621)

In [17]:
pred_input = tf.keras.Input(shape=(n_pred, task_dim[0], task_dim[1]))
y1 = tf.keras.layers.Flatten()(pred_input)
'''
condition_input = tf.keras.Input(shape=(3))
y2 = tf.keras.layers.Dense(8, activation='relu')(condition_input)
y = tf.keras.layers.Concatenate(axis=1)([y1, y2])
'''
y = tf.keras.layers.Dense(8, activation=tf.keras.layers.LeakyReLU())(y1)
y = tf.keras.layers.Dropout(0.8)(y)
y = tf.keras.layers.Dense(1, activation='sigmoid')(y)
discriminator = tf.keras.Model([pred_input], y)
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
tf.__version__

'2.8.0'

In [18]:
#reload(util_tools.cGAN_model)
from util_tools.cGAN_model import  Condition_GAN

In [19]:
cGAN = Condition_GAN(generator, discriminator, lr=0.000001)
cGAN.fit(1, 100, [X_high, X_low, X_ele, X_other], Y)

finish discriminator training
Epoch:1, batch:1/236, real_loss=1.107, fake_loss=0.390, g_loss=0.633
finish discriminator training
Epoch:1, batch:2/236, real_loss=1.259, fake_loss=0.380, g_loss=0.495
finish discriminator training
Epoch:1, batch:3/236, real_loss=1.342, fake_loss=0.151, g_loss=0.562
finish discriminator training
Epoch:1, batch:4/236, real_loss=2.989, fake_loss=0.583, g_loss=0.659
finish discriminator training
Epoch:1, batch:5/236, real_loss=1.352, fake_loss=0.526, g_loss=0.718
finish discriminator training
Epoch:1, batch:6/236, real_loss=1.291, fake_loss=1.061, g_loss=0.651
finish discriminator training
Epoch:1, batch:7/236, real_loss=2.735, fake_loss=1.187, g_loss=0.541
finish discriminator training
Epoch:1, batch:8/236, real_loss=1.387, fake_loss=0.224, g_loss=0.598
finish discriminator training
Epoch:1, batch:9/236, real_loss=1.317, fake_loss=0.172, g_loss=0.571
finish discriminator training
Epoch:1, batch:10/236, real_loss=1.387, fake_loss=0.655, g_loss=0.679
finish di

finish discriminator training
Epoch:1, batch:84/236, real_loss=0.834, fake_loss=1.059, g_loss=0.802
finish discriminator training
Epoch:1, batch:85/236, real_loss=0.661, fake_loss=0.680, g_loss=0.775
finish discriminator training
Epoch:1, batch:86/236, real_loss=0.532, fake_loss=0.782, g_loss=0.771
finish discriminator training
Epoch:1, batch:87/236, real_loss=0.808, fake_loss=0.680, g_loss=0.780
finish discriminator training
Epoch:1, batch:88/236, real_loss=0.534, fake_loss=0.643, g_loss=0.757
finish discriminator training
Epoch:1, batch:89/236, real_loss=0.797, fake_loss=0.828, g_loss=0.777
finish discriminator training
Epoch:1, batch:90/236, real_loss=0.768, fake_loss=0.763, g_loss=0.756
finish discriminator training
Epoch:1, batch:91/236, real_loss=0.602, fake_loss=0.651, g_loss=0.798
finish discriminator training
Epoch:1, batch:92/236, real_loss=0.767, fake_loss=0.726, g_loss=0.782
finish discriminator training
Epoch:1, batch:93/236, real_loss=0.675, fake_loss=0.882, g_loss=0.777


finish discriminator training
Epoch:1, batch:166/236, real_loss=0.678, fake_loss=0.682, g_loss=0.714
finish discriminator training
Epoch:1, batch:167/236, real_loss=0.641, fake_loss=0.633, g_loss=0.701
finish discriminator training
Epoch:1, batch:168/236, real_loss=0.681, fake_loss=0.629, g_loss=0.714
finish discriminator training
Epoch:1, batch:169/236, real_loss=0.711, fake_loss=0.740, g_loss=0.703
finish discriminator training
Epoch:1, batch:170/236, real_loss=0.701, fake_loss=0.723, g_loss=0.685
finish discriminator training
Epoch:1, batch:171/236, real_loss=0.702, fake_loss=0.670, g_loss=0.717
finish discriminator training
Epoch:1, batch:172/236, real_loss=0.663, fake_loss=0.657, g_loss=0.673
finish discriminator training
Epoch:1, batch:173/236, real_loss=0.685, fake_loss=0.793, g_loss=0.682
finish discriminator training
Epoch:1, batch:174/236, real_loss=0.648, fake_loss=0.610, g_loss=0.684
finish discriminator training
Epoch:1, batch:175/236, real_loss=0.619, fake_loss=0.716, g_l

In [20]:
generator.predict([X_high[-1:], X_low[-1:], X_ele[-1:], X_other[-1:]])

array([[[[0.91267836, 0.91228503, 0.91213405, 0.9122523 , 0.9117179 ],
         [0.9126853 , 0.9119334 , 0.9123945 , 0.91228676, 0.912086  ],
         [0.91253626, 0.91179043, 0.9122256 , 0.9113081 , 0.9117612 ],
         [0.9126473 , 0.9124548 , 0.9114045 , 0.91186583, 0.91165435],
         [0.91253436, 0.9130199 , 0.9113524 , 0.9112941 , 0.91221786]]]],
      dtype=float32)

In [25]:
X = [X_high, X_low, X_ele, X_other]
j =0
batch_size=2
batch_X = [d[j*batch_size:(j+1)*batch_size] for d in X]

In [28]:
batch_X[0].shape

(2, 4, 5, 5, 1)

In [30]:
np.zeros((batch_size, 1), dtype='int')

array([[0],
       [0]])

# 3. downscaler test

In [17]:
#reload(downscale)
from util_tools import downscale

In [18]:
dscler = downscale.downscaler(generator)

In [19]:
h_data = g_data[10:10+n_lag]
l_data = m_data[1836: 1856]
days = list(range(1836, 1856))

In [20]:
downscaled_data = dscler.downscale(h_data, l_data, ele_data,  [G_lats, G_lons, M_lats, M_lons], days, n_lag, n_pred, task_dim)

In [21]:
downscaled_data.shape

(5, 123, 207)

In [22]:
h_data.shape

(15, 123, 207)

In [25]:
downscaled_data[4].mean()

0.934871974800356

In [26]:
h_data[10+4].mean()

0.9450469764791418